In [2]:
import os
from fnmatch import fnmatch
import pandas as pd
import re
import tqdm
import plotnine as p9
import patchworklib as pw
import numpy as np

<Figure size 100x100 with 0 Axes>

In [ ]:
def dict_from_path(path: str):
    params_name_pattern = "([a-zA-Z]*)-"
    param_names = [path[m.start():m.end()-1] for m in re.finditer(params_name_pattern, path)]
    dict = {}
    for param in param_names:
        dict[param] = 0
    return dict

def construct_regex_string(path: str, names: dict):
    res: str = ""
    last_key = list(names.keys())[-1]
    for key in list(names.keys()):
        res = res + key + "-([0-9.]*)"
        if key != last_key:
            res = res + '_'
    return res

def extract_parms(filepath: str):
    param_st = filepath.split('/')[-2]
    params = dict_from_path(param_st)
    regex_str = construct_regex_string(param_st, params)
    result = re.search(regex_str, param_st)
    keys = list(params.keys())
    for x, key in enumerate(keys):
        params[key] = float(result.group(x+1))
    return params

In [ ]:
root = os.path.join(os.getcwd(),'output')
pattern = "logger_6_Ve.csv"
file_paths = []
dfsV = []

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            file_path = os.path.join(path, name)
            file_paths.append(file_path)
            df = pd.read_csv(file_path, sep='\t')
            params = extract_parms(file_path)
            add = True
            for key in list(params.keys()):
                if round(params[key],1) != 999.0 and round(params[key],1) % 0.2 != 0:
                    add = False
                    continue
                df[key] = round(params[key],1)
            if add == True:
                dfsV.append(df)

In [ ]:
simV = pd.concat(dfsV)
simV = simV[simV["10"] != 10]
simV['time'] = np.tile(np.repeat(range(0,51),10),int(len(simV.index)/510))
simV['sum_V_x'] = simV.iloc[:, 1:11].sum(axis=1)
simV = simV.drop(columns=[str(x) for x in range(0,11)])
simV = simV.groupby(list(simV.columns[:-1])).agg({'sum_V_x': 'sum'}).reset_index()
simV = simV.groupby(['DV','bcf','pi','cv','time']).agg({'sum_V_x':['mean','std']})
simV.columns = ["_".join(x) for x in simV.columns.ravel()]
simV = simV.rename({'sum_V_x_mean':'mean','sum_V_x_std':'std'}, axis='columns')
simV['cv_cat'] = simV['cv'].astype("category")
simV

In [ ]:
root = os.path.join(os.getcwd(),'output')
pattern = "logger_2.csv"
file_paths = []
dfs = []

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            file_path = os.path.join(path, name)
            file_paths.append(file_path)
            df = pd.read_csv(file_path, sep='\t')
            params = extract_parms(file_path)
            add = True
            for key in list(params.keys()):
                if round(params[key],1) != 999.0 and round(params[key],1) % 0.2 != 0:
                    add = False
                    continue
                df[key] = round(params[key],1)
            if add == True:
                dfs.append(df)

In [ ]:
sim = pd.concat(dfs)
sim = sim[sim['time'] >= 10]
simI = sim.groupby(['DV','bcf','pi','cv','time']).agg({'celltype.infected.size':['mean','std']})
simI.columns = ["_".join(x) for x in simI.columns.ravel()]
simI = simI.rename({'celltype.infected.size_mean':'mean','celltype.infected.size_std':'std'}, axis='columns')
simI = simI.reset_index()
#simI = simI.query('bcf > 0.8 and pi > 0.8 and cv > 0.8')
simI['cv_cat'] = simI['cv'].astype("category")
simI

In [ ]:
df_ode = pd.read_csv('simulations_ode.csv')